# BOtCH: Улучшенная версия чат-бота на Python

Этот ноутбук демонстрирует улучшенную реализацию чат-бота с модульной архитектурой, логированием, тестами, CLI-демо, метриками и возможной практической привязкой к FAQ-сценарию.

## 1. Импорт библиотек и конфигурация логирования

In [ ]:
import logging
from datetime import datetime
import re
import json
from collections import Counter

# Настройка логирования
logging.basicConfig(
    filename='botch_log.json',
    level=logging.INFO,
    format='%(message)s'
)

## 2. Модуль логики чат-бота

In [ ]:
class BotCH:
    def __init__(self):
        self.intents = {
            'greeting': [r'привет', r'здравствуй', r'добрый день'],
            'goodbye': [r'пока', r'до свидания'],
            'thanks': [r'спасибо', r'благодарю'],
            'howareyou': [r'как дела', r'как ты']
        }
        self.responses = {
            'greeting': 'Привет! Чем могу помочь?',
            'goodbye': 'До свидания!',
            'thanks': 'Пожалуйста!',
            'howareyou': 'У меня всё хорошо, спасибо за вопрос!'
        }

    def classify_intent(self, message):
        for intent, patterns in self.intents.items():
            for pattern in patterns:
                if re.search(pattern, message, re.IGNORECASE):
                    return intent
        return 'unknown'

    def get_response(self, message):
        intent = self.classify_intent(message)
        response = self.responses.get(intent, 'Извините, я не понимаю вас.')
        self.log_interaction(message, intent, response)
        return response

    def log_interaction(self, message, intent, response):
        log_entry = json.dumps({
            'timestamp': datetime.now().isoformat(),
            'message': message,
            'intent': intent,
            'response': response
        }, ensure_ascii=False)
        logging.info(log_entry)

## 3. Пример CLI-сценария

In [ ]:
def run_cli():
    bot = BotCH()
    print('BOtCH запущен. Введите сообщение или "выход" для завершения.')
    while True:
        user_input = input('Вы: ')
        if user_input.lower() == 'выход':
            print('BOtCH: До свидания!')
            break
        response = bot.get_response(user_input)
        print('BOtCH:', response)

# run_cli()  # Раскомментировать для запуска CLI

## 4. Анализ логов: частотный разбор интентов

In [ ]:
def analyze_logs(log_file='botch_log.json'):
    intents = []
    with open(log_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                entry = json.loads(line)
                intents.append(entry['intent'])
            except json.JSONDecodeError:
                continue
    return Counter(intents)

# print(analyze_logs())

## 5. Тестирование функций

In [ ]:
def test_classify_intent():
    bot = BotCH()
    assert bot.classify_intent('Привет!') == 'greeting'
    assert bot.classify_intent('Спасибо большое') == 'thanks'
    assert bot.classify_intent('Как дела?') == 'howareyou'
    assert bot.classify_intent('До свидания') == 'goodbye'
    assert bot.classify_intent('Что ты умеешь?') == 'unknown'
    print('Все тесты пройдены.')

# test_classify_intent()

## 6. Метрики и выводы

In [ ]:
# Оценка точности классификации
true_cases = {
    'Привет': 'greeting',
    'Как ты?': 'howareyou',
    'Спасибо за помощь': 'thanks',
    'До встречи': 'goodbye',
    'Что ты умеешь?': 'unknown'
}

bot = BotCH()
correct = sum(1 for q, expected in true_cases.items() if bot.classify_intent(q) == expected)
total = len(true_cases)
print(f'Точность: {correct}/{total} ({correct / total * 100:.0f}%)')

## 7. Практическое применение: FAQ для студентов

In [ ]:
faq_data = {
    'где найти расписание': 'Расписание находится на сайте в разделе "Образование".',
    'когда экзамен': 'Даты экзаменов указаны в расписании и в LMS.',
    'как получить справку': 'Справку можно заказать через личный кабинет студента.'
}

class FAQBot(BotCH):
    def get_response(self, message):
        for question, answer in faq_data.items():
            if question in message.lower():
                self.log_interaction(message, 'faq', answer)
                return answer
        return super().get_response(message)

# faq_bot = FAQBot()
# print(faq_bot.get_response('Где найти расписание?'))